[Table of Contents](00.00-Learning-ML.ipynb#Table-of-Contents) &bull; [&larr; *Chapter 2.03 - k-Nearest Neighbours*](02.03-k-Nearest-Neighbours.ipynb) &bull; [*Chapter 2.05 - ?* &rarr;](02.05-?.ipynb)

---

# Chapter 2.04 - Decision Trees

Decision trees introduction, benefits

## Forming a tree

Splitting strategy, continuous and categorical data

## Overfitting

training error > testing error

## Cross Validation

as a method to combat overfitting while making the most of the data

## Feature Reduction

by determining most important variables from the tree structure

## Implementing a Decision Tree

In [1]:
## todo

TODO

In [2]:
## todo

TODO

In [3]:
## todo

TODO

## What next?

Logistic regression

---

[Table of Contents](00.00-Learning-ML.ipynb#Table-of-Contents) &bull; [&larr; *Chapter 2.03 - k-Nearest Neighbours*](02.03-k-Nearest-Neighbours.ipynb) &bull; [*Chapter 2.05 - ?* &rarr;](02.05-?.ipynb)